In [1]:
#!pip install -q flwr[simulation] torch torchvision matplotlib wandb

In [2]:
from collections import OrderedDict
from typing import Dict, List, Optional, Tuple

import matplotlib.pyplot as plt

import numpy as np
import torch
from tqdm import tqdm
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
import albumentations as A
from torch.cuda.amp.grad_scaler import GradScaler
from torch.cuda.amp.autocast_mode import autocast
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset, DataLoader, random_split
import os
from PIL import Image
import flwr as fl
import torch.optim as optim
from flwr.common import Metrics
import wandb
from typing import List, Union
from flwr.common import Parameters, Scalar
from flwr.server.client_proxy import ClientProxy, FitRes

DEVICE = torch.device("cuda")  # Try "cuda" to train on GPU
print(
    f"Training on {DEVICE} using PyTorch {torch.__version__} and Flower {fl.__version__}"
)

C:\Users\Saeed\Documents\GitHub\flwr_simulation\flwr_unet\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Training on cuda using PyTorch 1.9.1+cu111 and Flower 1.4.0


#### Set up initial parameters

In [3]:
NUM_CLIENTS = 1
BATCH_SIZE = 6
IMAGE_HEIGHT = 240
IMAGE_WIDTH = 240
lr = 0.00001
epochs = 50

#### Data Transform Functions

In [4]:
# Define the transform
transform = A.Compose(
        [
            A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
            A.Rotate(limit=35, p=1.0),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.1),
            A.Normalize(
                mean=[0.0, 0.0, 0.0],
                std=[1.0, 1.0, 1.0],
                max_pixel_value=255.0,
            ),
            ToTensorV2(),
        ],
    )

# used to transform validation set
val_transforms = A.Compose(
    [
        A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
        A.Normalize(
            mean=[0.0, 0.0, 0.0],
            std=[1.0, 1.0, 1.0],
            max_pixel_value=255.0,
        ),
        ToTensorV2(),
    ],
)

#### Load Data

In [5]:
class WoundDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = os.listdir(image_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        img_path = os.path.join(self.image_dir, self.images[index])
        mask_path = os.path.join(self.mask_dir, self.images[index])
        image = np.array(Image.open(img_path).convert("RGB"))
        mask = np.array(Image.open(mask_path).convert("L"), dtype=np.float32)
        mask[mask == 255.0] = 1.0

        if self.transform is not None:
            augmentations = self.transform(image=image, mask=mask)
            image = augmentations["image"]
            mask = augmentations["mask"]
        return image, mask

# Load images and masks 
trainset = WoundDataset("../data/train_data_small/train_images/", "../data/train_data_small/train_masks/", transform=transform)
testset = WoundDataset("../data/validation_data/val_images/", "../data/validation_data/val_masks/", transform=val_transforms)


# Split training set into 10 partitions to simulate the individual dataset
partition_size = len(trainset) // NUM_CLIENTS
lengths = [partition_size] * NUM_CLIENTS
print(f"len(trainset)  {len(trainset) }")
print(f"partition_size: {partition_size}")
print(f"lengths: {lengths}")

datasets = random_split(trainset, lengths, torch.Generator().manual_seed(42))

# Split each partition into train/val and create DataLoader
trainloaders = []
valloaders = []
testloader = DataLoader(testset, batch_size=BATCH_SIZE)

for ds in datasets:
    len_val = len(ds) // 10  # 10 % validation set
    len_train = len(ds) - len_val
    lengths = [len_train, len_val]
    ds_train, ds_val = random_split(ds, lengths, torch.Generator().manual_seed(42))
    trainloaders.append(DataLoader(ds_train, batch_size=BATCH_SIZE, shuffle=True))
    #valloaders.append(DataLoader(ds_val, batch_size=BATCH_SIZE))
    valloaders.append(testloader)

#testloader = DataLoader(testset, batch_size=BATCH_SIZE)


len(trainset)  35
partition_size: 35
lengths: [35]


In [6]:
# how to add dropout in UNET architecture
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),
        )

    def forward(self, x):
        return self.conv(x)

class UNET(nn.Module):
    def __init__(
            self, in_channels=3, out_channels=1, features=[64, 128, 256, 512],
    ):
        super(UNET, self).__init__()
        self.ups = nn.ModuleList()
        self.downs = nn.ModuleList()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Down part of UNET
        for feature in features:
            self.downs.append(DoubleConv(in_channels, feature))
            in_channels = feature

        # Up part of UNET
        for feature in reversed(features):
            self.ups.append(
                nn.ConvTranspose2d(
                    feature*2, feature, kernel_size=2, stride=2,
                )
            )
            self.ups.append(DoubleConv(feature*2, feature))

        self.bottleneck = DoubleConv(features[-1], features[-1]*2)
        self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=1)

    def forward(self, x):
        skip_connections = []

        for down in self.downs:
            x = down(x)
            skip_connections.append(x)
            x = self.pool(x)

        x = self.bottleneck(x)
        skip_connections = skip_connections[::-1]

        for idx in range(0, len(self.ups), 2):
            x = self.ups[idx](x)
            skip_connection = skip_connections[idx//2]

            if x.shape != skip_connection.shape:
                x = TF.resize(x, size=skip_connection.shape[2:])

            concat_skip = torch.cat((skip_connection, x), dim=1)
            x = self.ups[idx+1](concat_skip)

        return self.final_conv(x)

In [7]:
def check_accuracy(loader, model, device="cuda"):
    """Test the network on the training set."""
    print("~~~~ In test ~~~~")
    criterion = torch.nn.BCEWithLogitsLoss()
    loss = 0
    num_correct = 0
    num_pixels = 0
    dice_score = 0
    iou_score  = 0
    result = []
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device).unsqueeze(1)
            preds = torch.sigmoid(model(x))
            loss += criterion(preds, y).item()
            preds = (preds > 0.5).float()
            num_correct += (preds == y).sum()
            num_pixels += torch.numel(preds)
            dice_score += (2 * (preds * y).sum()) / (
                (preds.sum() + y.sum()) + 1e-8
            )
            # Calculate IoU score
            intersection = (preds * y).sum()
            union = (preds + y).sum() - intersection
            iou_score += (intersection + 1e-8) / (union + 1e-8)

    num_batches = len(loader)
    loss /= num_batches
    acc = num_correct/num_pixels*100
    diceS = dice_score/num_batches
    iouS = iou_score/num_batches
    correct_pixel = num_correct
    total_pixel = num_pixels
    print(f"Loss = {loss}")
    print(f"IoU Score = {iouS}")
    print(f"Dice Score = {diceS}")
    print("~~~~~ Out of test ~~~~~")

    model.train()
    
    result = [acc.item(), diceS.item(), iouS.item(), loss, correct_pixel.item(), total_pixel]
    
    return result
  

def train(net, trainloader, epochs: int, lr, device="cuda"):
    """Train the network on the training set."""
    print("~~~~ In train ~~~~")
    criterion = torch.nn.BCEWithLogitsLoss() #loss_fn
    optimizer = torch.optim.Adam(net.parameters(), lr)
    scaler = GradScaler() #torch.cuda.amp.
    net.train()
    for epoch in range(epochs):
        print(f'epoch => {epoch}')
        print(len(trainloader))
        correct, total, epoch_loss = 0, 0, 0.0
        for images, labels in trainloader:
            # examine image integrity here
            images = images.to(DEVICE)
            labels = labels.float().unsqueeze(1).to(device=DEVICE)
            # forward
            with torch.cuda.amp.autocast():
                outputs = net(images)
                loss = criterion(outputs, labels)
        # backward
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        print("end of epoch")
    print("~~~~ Out of train ~~~~")

#### Utility Functions

In [8]:
def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    # Multiply accuracy of each client by number of examples used
    print("_____metrics_______")
    print(metrics)
    print(">>>>>>>>>>>")

    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics] # Original
    dice = [ num_examples *m["dice_score"] for num_examples, m in metrics]
    iouS = [num_examples * m["iouS"] for num_examples, m in metrics]
    loss = [num_examples * m["loss"] for num_examples, m in metrics]

    #accuracies = [m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]
    print("!!!accuracies ", str(accuracies))
    print("!!!dice ", str(dice))
    print("!!!examples ", str(examples))
    print("!!!iouS ", str(iouS))
    # Aggregate and return custom metric (weighted average)
    acc = sum(accuracies)/sum(examples)
    dice_score = sum(dice)/sum(examples)
    iou_score = sum(iouS)/sum(examples)
    loss = sum(loss)/sum(examples)

    #wandb.log({"acc": acc, "dice_score": dice_score, "iou_score": iou_score, "loss": loss})

    return {"accuracies": acc, "dice_score": dice_score, "iouS": iou_score, "loss": loss}

In [9]:
class SaveModelStrategy(fl.server.strategy.FedAvg):
    def aggregate_fit(
        self,
        server_round: int,
        results: List[Tuple[fl.server.client_proxy.ClientProxy, fl.common.FitRes]],
        failures: List[Union[Tuple[ClientProxy, FitRes], BaseException]],
    ) -> Tuple[Optional[Parameters], Dict[str, Scalar]]:
        """Aggregate model weights using weighted average and store checkpoint"""

        # Call aggregate_fit from base class (FedAvg) to aggregate parameters and metrics
        aggregated_parameters, aggregated_metrics = super().aggregate_fit(server_round, results, failures)

        if aggregated_parameters is not None:
            print(f"Saving round {server_round} aggregated_parameters...")

            # Convert `Parameters` to `List[np.ndarray]`
            aggregated_ndarrays: List[np.ndarray] = fl.common.parameters_to_ndarrays(aggregated_parameters)

            # Convert `List[np.ndarray]` to PyTorch`state_dict`
            params_dict = zip(net.state_dict().keys(), aggregated_ndarrays)
            state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
            net.load_state_dict(state_dict, strict=True)

            # Save the model
            torch.save(net.state_dict(), f"models/model_round_{server_round}.pth")

        return aggregated_parameters, aggregated_metrics

In [10]:
def save_predictions_as_imgs(
    loader, model, client_id, folder="prediction_images/", device="cuda"
):
    model.eval()
    for idx, (x, y) in enumerate(loader):
        x = x.to(device=device)
        with torch.no_grad():
            preds = torch.sigmoid(model(x))
            preds = (preds > 0.5).float()
        torchvision.utils.save_image(
            preds, f"{folder}/pred_{idx}_{client_id}.png"
        )
        torchvision.utils.save_image(y.unsqueeze(1), f"{folder}{idx}_{client_id}.png")

    model.train()

In [11]:
#get local params
def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]


#update local model with global params
def set_parameters(device, net, parameters: List[np.ndarray]):
    print("in outside SP !!!!!!!!")
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)


#### Flower Client defintions

In [12]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader, device, learning_rate, epochs):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader
        self.device = device
        self.epochs = epochs
        self.lr = learning_rate

    def set_parameters(self, parameters):
        print(f"[Client {self.cid}] set_parameters")
        # set_parameters(self.device, self.net, parameters)
        params_dict = zip(self.net.state_dict().keys(), parameters)
        state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
        net.load_state_dict(state_dict, strict=True)

    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return [val.cpu().numpy() for _, val in self.net.state_dict().items()]
        #return get_parameters(self.net)

    def fit(self, parameters, config):
        print(f"[Client {self.cid}] fit, config: {config}")
        self.set_parameters(parameters)
        train(self.net, self.trainloader, self.epochs, self.lr, self.device)
        return self.get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        self.set_parameters(parameters)
        #loss, accuracy = test(self.net, self.valloader)
        result = check_accuracy(self.valloader, self.net, self.device)
        # print some examples to a folderthu
        save_predictions_as_imgs(self.valloader, self.net, self.cid, folder="prediction_images/",device=self.device)
        #[acc.item(), diceS.item(), iouS.item(), loss, correct_pixel.item(), total_pixel]
        print('~~~ loss = ', str(result[3]))
        return float(result[3]), len(self.valloader), {"accuracy": result[0], "dice_score": result[1], "iouS": result[2], "loss": result[3]}

def client_fn(cid) -> FlowerClient:
    net = UNET().to(DEVICE)
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    lr = 1e-4
    epochs = 50
    print("~~~~client created~~~~")
    return FlowerClient(cid, net, trainloader, valloader, DEVICE, lr, epochs)

In [13]:
# Create an instance of the model and get the parameters

net = UNET().to(DEVICE)
params = get_parameters(net)

strategy = SaveModelStrategy(
    fraction_fit=1.0,
    fraction_evaluate=1.0,
    min_fit_clients=1,
    min_evaluate_clients=1,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(params),
    evaluate_metrics_aggregation_fn=weighted_average,
)

# Specify client resources if you need GPU (defaults to 1 CPU and 0 GPU)
client_resources = None
if DEVICE.type == "cuda":
    client_resources = {"num_gpus": 1}

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=10),  # Just three rounds
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-06-21 23:34:07,599 | app.py:149 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
2023-06-21 23:34:10,637	INFO worker.py:1625 -- Started a local Ray instance.
INFO flwr 2023-06-21 23:34:13,910 | app.py:183 | Flower VCE: Ray initialized with resources: {'memory': 7659960731.0, 'node:127.0.0.1': 1.0, 'CPU': 20.0, 'object_store_memory': 3829980364.0, 'GPU': 1.0}
INFO flwr 2023-06-21 23:34:13,911 | server.py:86 | Initializing global parameters
INFO flwr 2023-06-21 23:34:13,911 | server.py:269 | Using initial parameters provided by strategy
INFO flwr 2023-06-21 23:34:13,912 | server.py:88 | Evaluating initial parameters
INFO flwr 2023-06-21 23:34:13,912 | server.py:101 | FL starting
DEBUG flwr 2023-06-21 23:34:13,912 | server.py:223 | fit_round 1: strategy sampled 1 clients (out of 1)


(launch_and_fit pid=50268) ~~~~client created~~~~
(launch_and_fit pid=50268) [Client 0] fit, config: {}
(launch_and_fit pid=50268) [Client 0] set_parameters
(launch_and_fit pid=50268) ~~~~ In train ~~~~
(launch_and_fit pid=50268) epoch => 0
(launch_and_fit pid=50268) 6
(launch_and_fit pid=50268) end of epoch
(launch_and_fit pid=50268) epoch => 1
(launch_and_fit pid=50268) 6
(launch_and_fit pid=50268) end of epoch
(launch_and_fit pid=50268) epoch => 2
(launch_and_fit pid=50268) 6
(launch_and_fit pid=50268) end of epoch
(launch_and_fit pid=50268) epoch => 3
(launch_and_fit pid=50268) 6
(launch_and_fit pid=50268) end of epoch
(launch_and_fit pid=50268) epoch => 4
(launch_and_fit pid=50268) 6
(launch_and_fit pid=50268) end of epoch
(launch_and_fit pid=50268) epoch => 5
(launch_and_fit pid=50268) 6
(launch_and_fit pid=50268) end of epoch
(launch_and_fit pid=50268) epoch => 6
(launch_and_fit pid=50268) 6
(launch_and_fit pid=50268) end of epoch
(launch_and_fit pid=50268) epoch => 7
(launch_an

DEBUG flwr 2023-06-21 23:34:35,294 | server.py:237 | fit_round 1 received 1 results and 0 failures
WARNING flwr 2023-06-21 23:34:35,552 | fedavg.py:243 | No fit_metrics_aggregation_fn provided


Saving round 1 aggregated_parameters...


DEBUG flwr 2023-06-21 23:34:35,821 | server.py:173 | evaluate_round 1: strategy sampled 1 clients (out of 1)


...

_____metrics_______
[(2, {'accuracy': 98.26273345947266, 'dice_score': 0.0, 'iouS': 2.8508445604202848e-12, 'loss': 0.9607655107975006})]
>>>>>>>>>>>
!!!accuracies  [196.5254669189453]
!!!dice  [0.0]
!!!examples  [2]
!!!iouS  [5.7016891208405696e-12]
(launch_and_evaluate pid=51024) ~~~ loss =  0.9607655107975006
(launch_and_fit pid=37516) ~~~~client created~~~~
(launch_and_fit pid=37516) [Client 0] fit, config: {}
(launch_and_fit pid=37516) [Client 0] set_parameters
(launch_and_fit pid=37516) ~~~~ In train ~~~~
(launch_and_fit pid=37516) epoch => 0
(launch_and_fit pid=37516) 6
(launch_and_fit pid=37516) end of epoch
(launch_and_fit pid=37516) epoch => 1
(launch_and_fit pid=37516) 6
(launch_and_fit pid=37516) end of epoch
(launch_and_fit pid=37516) epoch => 2
(launch_and_fit pid=37516) 6
(launch_and_fit pid=37516) end of epoch
(launch_and_fit pid=37516) epoch => 3
(launch_and_fit pid=37516) 6
(launch_and_fit pid=37516) end of epoch
(launch_and_fit pid=37516) epoch => 4
(launch_and_fit p

DEBUG flwr 2023-06-21 23:37:27,977 | server.py:237 | fit_round 4 received 1 results and 0 failures


Saving round 4 aggregated_parameters...


DEBUG flwr 2023-06-21 23:37:28,678 | server.py:173 | evaluate_round 4: strategy sampled 1 clients (out of 1)
(launch_and_evaluate pid=29384) 2023-06-21 23:37:30,649	ERROR serialization.py:387 -- [WinError 1455] The paging file is too small for this operation to complete. Error loading "C:\Users\Saeed\Documents\GitHub\flwr_simulation\flwr_unet\lib\site-packages\torch\lib\caffe2_detectron_ops_gpu.dll" or one of its dependencies.
(launch_and_evaluate pid=29384) Traceback (most recent call last):
(launch_and_evaluate pid=29384)   File "C:\Users\Saeed\Documents\GitHub\flwr_simulation\flwr_unet\lib\site-packages\ray\_private\serialization.py", line 385, in deserialize_objects
(launch_and_evaluate pid=29384)     obj = self._deserialize_object(data, metadata, object_ref)
(launch_and_evaluate pid=29384)   File "C:\Users\Saeed\Documents\GitHub\flwr_simulation\flwr_unet\lib\site-packages\ray\_private\serialization.py", line 268, in _deserialize_object
(launch_and_evaluate pid=29384)     return se

History (loss, distributed):
	round 1: 0.9545530378818512
	round 2: 0.9681576788425446
	round 3: 0.9607655107975006
History (metrics, distributed, evaluate):
{'accuracies': [(1, 98.26273345947266), (2, 1.7372684478759766), (3, 98.26273345947266)], 'dice_score': [(1, 0.0), (2, 0.03241487964987755), (3, 0.0)], 'iouS': [(1, 2.8508445604202848e-12), (2, 0.01648148149251938), (3, 2.8508445604202848e-12)], 'loss': [(1, 0.9545530378818512), (2, 0.9681576788425446), (3, 0.9607655107975006)]}